# get_freq_mutations test Endometrial

In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import CPTAC.Endometrial as en

In [27]:
def get_frequently_mutated(somatic_df, omics_mutations_df, cutoff=.1):  
    unique_genes = somatic_df['Gene'].unique() # get series of all mutated genes
    freq_mutated = []
    
    # get total tumor patients
    tumors = omics_mutations_df.loc[omics_mutations_df['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors) 
    
    for gene in unique_genes:
        gene_mutated = somatic_df.loc[somatic['Gene'] == gene]
        num_gene_mutated = len(gene_mutated)
        if (num_gene_mutated / total_tumor_patients) > cutoff:
            freq_mutated.append(gene)
        
    return freq_mutated

In [28]:
#get df
somatic = en.get_mutations()
clinical = en.get_clinical()
prot = en.get_proteomics()
gene = 'PTEN'
omics_mutations = en.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)

In [29]:
list1 = get_frequently_mutated(somatic, omics_mutations, .25)

In [33]:
print(len(list1))
list1

75


['ARID1A',
 'NEB',
 'LRP2',
 'PIK3CA',
 'FAT4',
 'DNAH5',
 'ADGRV1',
 'APC',
 'NSD1',
 'DST',
 'SYNE1',
 'MAP3K4',
 'DNAH11',
 'KMT2C',
 'ZFHX4',
 'PTEN',
 'LRRK2',
 'RYR3',
 'CTCF',
 'KMT2B',
 'ADGRG4',
 'USH2A',
 'DNAH14',
 'IGSF10',
 'KRAS',
 'DNAH10',
 'PIK3R1',
 'COL6A3',
 'CTNNB1',
 'FAT2',
 'PCLO',
 'SPTA1',
 'KMT2D',
 'HERC2',
 'ZFHX3',
 'RNF213',
 'OBSCN',
 'APOB',
 'FSIP2',
 'DNAH7',
 'ZDBF2',
 'ANK2',
 'DCHS2',
 'DNAH8',
 'EYS',
 'REV3L',
 'ABCA13',
 'PTPRZ1',
 'RP1',
 'CSMD3',
 'ANK3',
 'DYNC2H1',
 'SACS',
 'CCDC168',
 'AHNAK2',
 'WDR87',
 'DMD',
 'ATRX',
 'TENM1',
 'HMCN1',
 'DOCK3',
 'BRWD3',
 'MUC16',
 'HUWE1',
 'TENM3',
 'LYST',
 'RYR2',
 'LRP1B',
 'XIRP2',
 'SI',
 'MUC17',
 'FAT3',
 'SYNE2',
 'DNAH3',
 'INPPL1']

In [22]:
# check genes in freq_mutated list
gene = 'ARID1A'
total_tumor_patients = 97
cutoff = .1
gene_mutated = somatic.loc[somatic['Gene'] == gene]
num_gene_mutated = len(gene_mutated)
print(num_gene_mutated, ' / ', total_tumor_patients, ' ', (num_gene_mutated / total_tumor_patients))
if (num_gene_mutated / total_tumor_patients) > cutoff:
    print('true')
else:
    print('false')

65  /  97   0.6701030927835051
true


In [25]:
unique_genes = somatic['Gene'].unique()
print('unique_genes: ', len(unique_genes))
not_in_proteomics = []
in_proteomics = []

unique_genes:  14814


In [15]:
#challenges with append_mutations_to_omics, can't merge with somatic (prot vs gene names)
#prepare list for .append_mutations_to_omics
for gene in unique_genes:
    if gene in proteomics.columns:
        in_proteomics.append(gene)
    else:
        not_in_proteomics.append(gene)

In [20]:
len(in_proteomics) #8265
len(not_in_proteomics) #6549

6549

In [60]:
#option 1:get total patients
total = clinical.loc[clinical[#'#fixme'] == 'Tumor']
print(len(total)) #95 for endometrial

    #colon
#total = clinical.loc[clinical['Tumor.Status'] == 'With tumor']
#print(len(total)) #16 with tumor for colon

16


attrib_name,Age,Gender,Mucinous,Subsite,pathalogy_T_stage,pathalogy_N_stage,Stage,CEA,Vascular_Invasion,Lymphatic_Invasion,...,Vital.Status,Tumor.Status,Mutation_Phenotype,StromalScore,ImmuneScore,ESTIMATEScore,TumorPurity,immuneSubtype,CIN,Integrated.Phenotype
01CO006,904,Female,Mucinous,Ascending Colon,T4a,N2b,Stage III,NaN,Yes,Yes,...,Living,With tumor,MSS,466.1746,1542.0233,2008.198,0.6218,2,CIN.low,EMT
01CO008,652,Female,Mucinous,Descending Colon,T3,N0,Stage II,NaN,No,No,...,Living,With tumor,MSS,-1122.8413,-54.7177,-1177.559,0.9081,1,CIN.high,Epithelial
01CO014,930,Female,Mucinous,Hepatix Flexure,T3,N2a,Stage III,3.6,No,No,...,Living,With tumor,MSI-H,-824.9369,636.0912,-188.8456,0.838,3,CIN.high,Hypermutated
01CO019,757,Female,Mucinous,Sigmoid Colon,T4a,N2a,Stage III,3.5,Yes,Yes,...,Living,With tumor,MSS,-735.6004,-168.7173,-904.3177,0.8905,1,CIN.high,Epithelial
05CO033,737,Male,Mucinous,Cecum,T2,N2a,Stage III,1.8,Yes,Yes,...,Living,With tumor,MSS,1082.1732,1782.7777,2864.951,0.5186,2,CIN.low,EMT


In [28]:
#option 2: trying to use append_mutations_to_omics to get total patients
gene = 'PTEN'
omics_mutations = en.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)
tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
total_tumor_count = len(tumors)
print(total_tumor_count)

95


In [43]:
#count mutated
gene1 = somatic.loc[somatic['Gene'] == 'PIK3R1']
count = len(gene1)
count

46

In [44]:
# test for loop to count genes
total_tumor_patients = 95
cutoff = .1
freq_mutated = []
for gene in unique_genes:
    gene_mutated = somatic.loc[somatic['Gene'] == gene]
    num_gene_mutated = len(gene_mutated)
    if (num_gene_mutated / total_tumor_patients) > cutoff:
            freq_mutated.append(gene)

In [46]:
len(freq_mutated)

814